>AlexNet - f1-score = 0.728075 with Normalize

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.notebook import tqdm

import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import lr_scheduler
from torchvision.datasets import ImageFolder, DatasetFolder
from torchvision.datasets.folder import default_loader
from torchvision.models import alexnet
import torchvision.transforms as transforms

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'pandas'

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

#### Preparation train and test date

In [5]:
train_data_path = './data/journey-springfield/train/simpsons_dataset'
test_data_path = './data/journey-springfield/testset'
train_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_image_folder = ImageFolder(root=train_data_path, transform=train_transform)
test_image_folder = ImageFolder(root=test_data_path, transform=test_transform)

In [11]:
train_size = 0.8
valid_size = 0.2
train_dataset, valid_dataset = random_split(train_image_folder, [train_size, valid_size])

In [6]:
batch_size=20
# train_loader = DataLoader(dataset=train_image_folder, batch_size=batch_size, shuffle=True, num_workers=4)
# valid_loader = DataLoader(dataset=test_image_folder, shuffle=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(valid_dataset)

#### NN

In [7]:
def plot_trainig(train_losses, valid_loss):
    plt.figure(figsize=(7, 3))
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.plot(train_losses, label='train_loss')
    plt.plot(valid_losses, label='valid_loss')
    plt.legend()
    plt.show()

In [9]:
# class ImageNet(nn.Module):
#     def __init__(self):
#         super(ImageNet, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4)
#         self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2)
#         self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1)
#         self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, padding=1)
#         self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1)
#         self.fc1 = nn.Linear(in_features=6400, out_features=4096)
#         self.fc2 = nn.Linear(in_features=4096, out_features=4096)
#         self.fc3 = nn.Linear(in_features=4096, out_features=42)
        
#     def forward(self, x):
#         x = F.relu(self.conv1(x), inplace=True)
#         x = F.max_pool2d(x, kernel_size=3, stride=2)
#         x = F.relu(self.conv2(x), inplace=True)
#         x = F.max_pool2d(x, kernel_size=3, stride=2)
#         x = F.relu(self.conv3(x), inplace=True)
#         x = F.relu(self.conv4(x), inplace=True)
#         x = F.relu(self.conv5(x), inplace=True)
#         x = F.max_pool2d(x, kernel_size=3, stride=2)
#         x = torch.flatten(x, 1)
#         x = F.dropout(x, p=0.5)
#         x = F.relu(self.fc1(x), inplace=True)
#         x = F.dropout(x, p=0.5)
#         x = F.relu(self.fc2(x), inplace=True)
#         x = self.fc3(x)
#         return x

AlexNet = alexnet()
AlexNet.classifier[4] = nn.Linear(4096, 1024)
AlexNet.classifier[6] = nn.Linear(1024, 42)
AlexNet = AlexNet.to(device)

In [10]:
def fit(epochs, model, loss_func, opt, train_dl, scheduler):
    train_losses = []
    # valid_losses = []
    # valid_f1 = []

    for epoch in range(epochs):
        model.train()
        loss_sum = 0
        for xb, yb in tqdm(train_dl, desc=f'{epoch+1} st train loop'):
            xb, yb, = xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = loss_func(model(xb), yb)
            loss_sum += loss.item()
            loss.backward()
            opt.step()
        train_losses.append(loss_sum / len(train_dl))
        scheduler.step()

        # model.eval()
        # loss_sum = 0
        # predictions = []
        # true_labels = []
        # with torch.no_grad():
        #     for x, y in tqdm(valid_dl, desc=f'{epoch+1} st valid loop'):
        #         probs = model(x)
        #         loss_sum += loss_func(probs, y).item()
        #         y_pred = torch.argmax(probs, dim=-1)
        #         true_labels.append(y.item())
        #         predictions.append(y_pred.item())
        # valid_losses.append(loss_sum / len(valid_dl))
        # f1 = f1_score(true_labels, predictions, average='weighted')
        # valid_f1.append(f1)

        # print(f'Epoch {epoch+1}: Train Loss = {train_losses[-1]}, Valid Loss = {valid_losses[-1]}, F1-score = {f1}')
        print(f'Epoch {epoch+1}: Train Loss = {train_losses[-1]},  Lr = {scheduler.get_last_lr()}')

    return train_losses


loss_func = nn.CrossEntropyLoss()
opt = optim.Adam(AlexNet.parameters(), lr=0.001)
scheduler = lr_scheduler.StepLR(opt, step_size=7, gamma=0.1)

In [11]:
epochs = 10
info = fit(epochs, AlexNet, loss_func, opt, train_loader, scheduler)
plot_trainig(info)

1 st train loop:   0%|          | 0/2094 [00:00<?, ?it/s]

Epoch 1: Train Loss = 3.1384564964910178,  Lr = [0.001]


2 st train loop:   0%|          | 0/2094 [00:00<?, ?it/s]

Epoch 2: Train Loss = 3.1208700817202883,  Lr = [0.001]


3 st train loop:   0%|          | 0/2094 [00:00<?, ?it/s]

Epoch 3: Train Loss = 3.116413153686633,  Lr = [0.001]


4 st train loop:   0%|          | 0/2094 [00:00<?, ?it/s]

Epoch 4: Train Loss = 3.115189084350892,  Lr = [0.001]


5 st train loop:   0%|          | 0/2094 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [43]:
class_names = train_dataset.classes
y_pred = []
model.eval()
with torch.no_grad():
    for x, _ in test_loader:
        output = model(x)
        probs = F.softmax(output, dim=-1)
        _, y_pred_class = torch.max(probs, dim=-1)
        y_pred_name_class = class_names[y_pred_class]
        y_pred.append(y_pred_name_class)

In [44]:
df_sub = pd.read_csv('./data/journey-springfield/sample_submission.csv')
df_sub['Expected'] = y_pred
df_sub.to_csv('df_sub.csv', index=False)
df_sub

,Id,Expected
0,img0.jpg,moe_szyslak
1,img1.jpg,charles_montgomery_burns
2,img2.jpg,ned_flanders
3,img3.jpg,chief_wiggum
4,img4.jpg,apu_nahasapeemapetilon
...,...,...
986,img986.jpg,nelson_muntz
987,img987.jpg,apu_nahasapeemapetilon
988,img988.jpg,charles_montgomery_burns
989,img989.jpg,chief_wiggum
